In [1]:
!pip install psycopg2

In [1]:
import psycopg2

import sys
sys.path.append('../../')
from src.pySQL_funcs import pretty_query

with open('../../src/pw') as pw_file:
    pw = pw_file.readline()

LINK TO DATABASE INFO & SCHEMA: https://github.com/isaac-campbell-smith/Pokestars

In [2]:
conn = psycopg2.connect(pw)
cur = conn.cursor()

### OVERARCHING QUESTION
Volcarona is one of my all-time favorite Pokemon - a powerful fire, bug type reminiscent of Mothra with some of the highest base stat totals in the competitively legal Pokedex. Unfortunately in the first 3 generations since it's release it suffered a fate shared by Charizard, Butterfree and Articuno known as 'Stealth Rock Syndrome'. In the most recent generation however, a new item was released that has helped make Volcarona a Top-10 threat in the meta-game. Today we'll be diving into identify what that item is, quantify how much it boosted Volcarona's usage, and look at who else benefited from this new item. 

### QUESTION 1.a

Start by writing a query to identify the name of this item. Serendipitously, it has the highest item count. A starter table containing Volcarona's usage and every item detail for all months has been provided (in lieu of an actually simple warm-up)
<br><br>
Your output table should include all the columns in the starter table:
<br>
usage (from the Battles table), item_name, item_count, month
<br><br>
[Solution](#Q1.a)

In [3]:
query = """
SELECT b.usage, i.name AS item_name, 
       bi.count AS item_count, b.month
  FROM battles AS b 
  JOIN pokemon AS p ON b.id=p.id
  JOIN battle_items AS bi ON b.id=bi.id
                AND b.month=bi.month
  JOIN items i ON bi.item_id=i.id
 WHERE p.name='Volcarona'
ORDER BY TO_DATE(b.month, 'MM YYYY')
;
"""
pretty_query(cur, query, conn)

,usage,item_name,item_count,month
0,0.039162,armorfossil,23.0,08-2015
1,0.039162,scopelens,120.0,08-2015
2,0.039162,figyberry,7.0,08-2015
3,0.039162,zoomlens,72.0,08-2015
4,0.039162,blackbelt,17.0,08-2015
...,...,...,...,...
5729,0.185429,occaberry,20.0,09-2020
5730,0.185429,sitrusberry,1646.0,09-2020
5731,0.185429,cobaberry,18.0,09-2020
5732,0.185429,masterball,2.0,09-2020


### QUESTION 1.b
Let's look at it from a slightly different metric. Get the item that appeared in the highest percentage of all Volcarona sets for a single month. Include all the same columns from 1.a but replace item_count w/ item_pct.<br><i>***HINT***</i> Before you try to do some overly complicated summation formula, double check the columns available to you in the battles table.<br><br>
[Solution](#Q1.b)

In [4]:
query = """
;
"""
pretty_query(cur, query, conn)

### QUESTION 2.a
Now I'd like to know what month had the biggest increase from the previous month in Volcarona's usage. You'll need to implement a LAG function to accomplish this. LAG is another WINDOW function (like RANK). Unlike RANK, the main column argument is passed in the first set of parentheses (RANK generally relies on the ORDER BY argument within the OVER parentheses). LAG returns a column with the same values of the corresponding column, offset by a specific interval.
<br><br>
The general syntax is:<br>
> SELECT LAG(column, offset (<i>int</i>)) OVER()

Note that the OVER parentheses are left blank here. Like the RANK function we looked at last week, PARTITION BY clauses can be and often are used here.  While you shouldn't need to use one to execute this query, LAG, RANK and other similar WINDOW functions will throw an error if you exclude the OVER() statement.

FOR EXAMPLE:<br>

| nums |
|:----:|
| 1 |
| 2 |
| 3 |
| 4 |
| 5 |

> SELECT nums, LAG(nums, 1) OVER() <br>
&nbsp;&nbsp;   FROM table;<br>

OUTPUT: <br>

| nums | lag |
|:----:|:----:|
| 1 | null |
| 2 | 1 |
| 3 | 2 |
| 4 | 3 |
| 5 | 4 |

> SELECT nums, LAG(nums, 3) OVER() <br>
&nbsp;&nbsp;   FROM table;<br>

OUTPUT: <br>

| nums | lag |
|:----:|:----:|
| 1 | null |
| 2 | null |
| 3 | null |
| 4 | 1 |
| 5 | 2 |

[Solution](#Q2)

In [7]:
query = """
;
"""
pretty_query(cur, query, conn)

### QUESTION 2.b

Let's revisit our RANK OVER function again and combine it with LAG. Which month did Volcarona see it's largest jump in the usage rankings from the previous month? <br><br>
As a general thought exercise (which you're free to skip), think about out why this month is not the same as the month from question 2.a (spoiler lol) and which metric is more valid for understanding the importance of heavydutyboots for Volcarona's competitive viability. 

In [8]:
query = """
;
"""
pretty_query(cur, query, conn)

### QUESTION 3

There are 3 other pre-generation 8 Pokemon that greatly benefited from the addition of heavydutyboots. These 3 Pokemon utilized the item as their number 1 or 2 option and had a more than 5% change in their usage at the start of the current meta-game. Name these Pokemon. Try to incorporate the LAG function if you can (the alternative is more efficient though it will require a bit more code).  

<br>
<i>NOTE</i>: Generation 8 meta-game officially started in December 2019. 
<br><br>

[Solution](#Q3)

In [9]:
#USING LAG
query = """
;
""" 
pretty_query(cur, query, conn)

# RUN THIS CELL WHEN YOU'RE DONE OR ELSE I WILL FIND YOU AND HURT YOU

In [16]:
cur.close()  # Close the cursor
conn.close()

# SOLUTIONS

# Q1.a

In [10]:
query = """
WITH volcarona AS
            (SELECT b.usage, i.name AS item_name, 
                    bi.count AS item_count, b.month

              FROM battles b JOIN pokemon p ON b.id=p.id
              JOIN battle_items bi ON b.id=bi.id AND b.month=bi.month
              JOIN items i ON bi.item_id=i.id
              WHERE p.name='Volcarona'
              ORDER BY TO_DATE(b.month, 'MM YYYY'))
SELECT *
  FROM volcarona
  WHERE item_count=(SELECT MAX(item_count) FROM volcarona)
;
"""
pretty_query(cur, query, conn)

,usage,item_name,item_count,month
0,0.192257,heavydutyboots,559531.0,08-2020


# Q1.b

In [11]:
query = """
WITH volcarona AS
            (SELECT b.usage, i.name AS item, 
                    bi.count/b.count AS item_pct, b.month

              FROM battles b JOIN pokemon p ON b.id=p.id
              JOIN battle_items bi ON b.id=bi.id AND b.month=bi.month
              JOIN items i ON bi.item_id=i.id
              WHERE p.name='Volcarona'
              ORDER BY TO_DATE(b.month, 'MM YYYY'))
SELECT usage, item, item_pct, month
  FROM volcarona
  WHERE item_pct=(SELECT MAX(item_pct) FROM volcarona)
;
"""
pretty_query(cur, query, conn)

,usage,item,item_pct,month
0,0.110028,heavydutyboots,0.870265,06-2020


# Q2.a

In [17]:
query = """
WITH volcarona 
  AS 
    (SELECT month, b.usage - LAG(b.usage, 1) OVER() AS diff
      FROM battles AS b
      JOIN pokemon AS p 
        ON b.id=p.id
      WHERE p.name='Volcarona')
SELECT diff, month FROM volcarona
  WHERE diff=(SELECT MAX(diff) 
                FROM volcarona)
;
"""
pretty_query(cur, query, conn)

InterfaceError: connection already closed

In [15]:
query = """
SELECT month, b.usage - LAG(b.usage, 1) OVER() AS diff
      FROM battles AS b
      JOIN pokemon AS p 
        ON b.id=p.id
      WHERE p.name='Volcarona'
      ORDER BY diff DESC 
      LIMIT 1 OFFSET 1
;
"""
pretty_query(cur, query, conn)

,month,diff
0,07-2020,0.052575


# Q2.b

In [13]:
query = """
WITH 
    volcarona 
  AS 
    (SELECT month, rank, LAG(rank, 1) OVER() - rank AS diff 
      FROM (SELECT month, id, 
             RANK() OVER (PARTITION BY month ORDER BY usage DESC)
              FROM battles
            ORDER BY TO_DATE(month, 'MM YYYY')
             ) t
      JOIN pokemon AS p 
        ON t.id=p.id
      WHERE p.name='Volcarona'
      )
      
SELECT month
  FROM volcarona
 WHERE diff=(SELECT MAX(diff) FROM volcarona)
;
"""
pretty_query(cur, query, conn)

,month
0,12-2016


Conceptual:<br>
If you have a look at the ranking and usage (for any month really) there's a pretty steep cut cutoff outside of the very top Pokemon and eventually start to logjam asymptotically, so if a Pokemon were to climb from rank 60 to 50, it wouldn't take as much of a usage change as it would to get from 10 to 5. That's essentially what happened here and why changes in ranking aren't as insightful as change in usage (Volcarona climbed from 37 to 28 in Nov-Dec 2016 with a .009 usage increase, but 11 to 6 this summer with a .08 usage increase). Perhaps a more appropriate way to frame how it's rank has been affected by access to heavy duty boots is it's average ranking with and without heavy duty boots.

# Q3

In [14]:
#USING LAG
query = """
WITH vt AS
          (
           SELECT p.name, b.usage, i.name AS item, bi.count, b.month,
                  RANK() OVER(PARTITION BY bi.month, p.name ORDER BY bi.count DESC) AS item_rank
             FROM battles b JOIN pokemon p ON b.id=p.id
             JOIN battle_items bi ON b.id=bi.id AND b.month=bi.month
             JOIN items i ON bi.item_id=i.id
            WHERE p.name != 'Volcarona'
              AND b.month IN ('11-2019', '12-2019')
           )
SELECT name
  FROM
      (
       SELECT name, usage, item, month,
              usage - 
              LAG(usage, 2) OVER(PARTITION BY name ORDER BY month) AS diff 
         FROM vt WHERE item_rank<=2
       ) t
       
 WHERE diff > 0.05 AND item='heavydutyboots'
;
""" 
pretty_query(cur, query, conn)

,name
0,Gyarados
1,Mandibuzz
2,Rotom-Heat


In [15]:
#ALTERNATIVE AND PERHAPS MORE EFFICIENT
query = """
SELECT t1.name
  FROM
      (
           SELECT p.name, b.usage, b.month
             FROM battles b JOIN pokemon p ON b.id=p.id
            WHERE b.month ='11-2019'
      ) t1
  JOIN 
      (
           SELECT * 
             FROM 
                (
                     SELECT p.name, b.usage, i.name AS item, 
                            RANK() OVER(PARTITION BY p.name ORDER BY bi.count DESC) AS item_rank, 
                            bi.count, b.month
                       FROM battles b JOIN pokemon p ON b.id=p.id
                       JOIN battle_items bi ON b.id=bi.id 
                                           AND b.month=bi.month
                       JOIN items i ON bi.item_id=i.id
                        WHERE b.month ='12-2019'
                 ) t 
            WHERE item_rank<=2 
              AND item='heavydutyboots'
        ) t2
          
    ON t1.name=t2.name
 WHERE t2.usage - t1.usage > 0.05 
;
"""
pretty_query(cur, query, conn)

,name
0,Gyarados
1,Mandibuzz
2,Rotom-Heat
